In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

KeyboardInterrupt: 

In [ ]:
fname = '~/Documents/projects/thesis/results/output/compiled_SUPERsnow.nc'

ds = xr.open_dataset(fname,  chunks={'time':1,'x':1000,'y':1000})
ds.close()
ds.nbytes / 1e9

In [ ]:
#Select peak snow dates
dsmall = ds.sel(time='2013')
dsmall.close()

ds = dsmall
ds.close()
ds

# '''peak snow depth dates....(?)'''
# flist = list(flist[i] for i in (0,7,22,28,41,49))
# dates = list(dates[i] for i in (0,7,22,28,41,49))

In [ ]:
# plt.imshow(ds.snow[0]/1000, cmap='coolwarm'); plt.colorbar(); plt.title('One snow depth image'); plt.show() #zeros outside of mask because nans=float, 0=ints
# plt.imshow(ds.dem, cmap='terrain'); plt.colorbar(); plt.title('DEM'); plt.show()
# plt.imshow(ds.veg, cmap='Greens'); plt.colorbar(); plt.title('Veg Height'); plt.show()

In [ ]:
#std over 2013
std=ds.snow.std(dim='time')
std = np.where(std>.001, std, np.nan)

fig = plt.figure(figsize=(15, 10))
plt.imshow(std/1000, cmap='jet')
plt.colorbar()
plt.title('Standard deviation [m] snow depth 2013 (6 flights)')
plt.show()

In [ ]:
#sd above treeline

# d = ds.snow.where(ds.snow>0)
# print(d)
sd_above_treeline = ds.snow.where(ds.dem > 2900)

fig = plt.figure(figsize=(15, 10))
plt.imshow(sd_above_treeline[3], cmap='jet')
plt.colorbar()
plt.title('Snow Depth [mm] above 2900m one time 2013')

In [ ]:
sd_above_treeline

#### standardize snow depth function

In [ ]:
## MM working


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  standardize lidar

def standard(x):
    a = np.where(x>0, x, np.nan)
    return x/np.nanmean(a)

s = xr.apply_ufunc(standard, ds.snow, dask='parallelized', output_dtypes=[np.int16])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  compute normalized stand dev for peak SWE dates
s_std=np.nanstd(s,axis=0) #np language

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.imshow(np.where(s_std>0,s_std,np.nan),cmap='jet', vmax=1)
plt.colorbar()
plt.title('Standard deviation snow depth 2013 (6 flights)')
plt.show()

In [ ]:
ds

#### histogram attempt with np

In [ ]:
ds.nbytes / 1e9
ds


In [ ]:
# histogram function
def histogram(x):
    a = np.where(x>0, x, np.nan)
    return np.histogram(a, bins=50)

s = xr.apply_ufunc(histogram, ds.isel(time=1).snow, dask='parallelized', output_dtypes=[np.int16] )

In [ ]:
s

In [ ]:
## MM with Charlie

# # Normalized variance for flights closest to peak SWE dates
# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# # 1). mean snow depth for each year (get 6 means for each year, 2013-2018 (compute without zeros ideally))
# # means=ds.snow.mean(dim=('x', 'y')) #ds.mean(dim=('x','y')--mean over all layers)
# # means

# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# # 2). standardize lidar

# def standard(x):
#     a = np.where(x>0, x, np.nan)
#     return x/np.nanmean(a)

# s = xr.apply_ufunc(standard, ds.snow)

# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# # 3). compute normalized variance for peak SWE dates
# s_var=np.nanvar(s,axis=0) #np language